In [ ]:
import pandas as pd
import requests
import json
import time
from typing import List, Dict
import subprocess
import signal
import sys
import logging
from datetime import datetime
import pytz

# Configure logging
ist = pytz.timezone('Asia/Kolkata')
ist_time = datetime.now(ist)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'llm_evaluation_{ist_time.strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)

# Constants
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODELS = [
    # "hf.co/bartowski/Qwen2.5-7B-Instruct-GGUF:Q5_K_L",
    # "hf.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF:Q5_K_L",
    # "hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:F16",
    # "hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:F16",
    # "hf.co/bartowski/Qwen2.5-14B-Instruct-GGUF:Q2_K",
    # "hf.co/bartowski/Qwen2.5-3B-Instruct-GGUF:F16",
    # "hf.co/bartowski/Qwen2.5-1.5B-Instruct-GGUF:F16",
    "hf.co/bartowski/Phi-3-medium-128k-instruct-GGUF:Q3_K_S",
    "hf.co/RichardErkhov/princeton-nlp_-_gemma-2-9b-it-SimPO-gguf:Q4_K_M",
]

def load_questions() -> pd.DataFrame:
    """Load questions from CSV file."""
    try:
        df = pd.read_csv('/home/ubuntu/quantumLeap/quantumEval_claude.csv')
        logging.info(f"Successfully loaded {len(df)} questions from CSV")
        return df
    except Exception as e:
        logging.error(f"Error loading questions: {e}")
        sys.exit(1)

def start_ollama_model(model_name: str) -> subprocess.Popen:
    """Start an Ollama model."""
    try:
        process = subprocess.Popen(
            ['ollama', 'run', model_name],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        time.sleep(10)  # Wait for model to initialize
        logging.info(f"Started model: {model_name}")
        return process
    except Exception as e:
        logging.error(f"Error starting model {model_name}: {e}")
        return None

def stop_ollama_model(process: subprocess.Popen):
    """Stop the Ollama model process."""
    if process:
        process.send_signal(signal.SIGTERM)
        process.wait()
        logging.info("Stopped Ollama model")

def query_model(question: str, model_name: str) -> str:
    """Query the model and get response."""
    prompt = f"Please answer the following question about Clotaire Rapaille's Culture Code methodology:\n\n{question}\n\nProvide a clear and concise answer based on Rapaille's work."
    
    payload = {
        "model": model_name,
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post(OLLAMA_API_URL, json=payload)
        response.raise_for_status()
        return response.json()['response']
    except Exception as e:
        logging.error(f"Error querying model {model_name}: {e}")
        return f"ERROR: {str(e)}"

def evaluate_models(questions_df: pd.DataFrame) -> pd.DataFrame:
    """Evaluate all models on all questions."""
    results = []
    
    for model in MODELS:
        logging.info(f"Starting evaluation for model: {model}")
        
        # Start the model
        process = start_ollama_model(model)
        if not process:
            continue
            
        try:
            for _, row in questions_df.iterrows():
                logging.info(f"Processing Question {row['Question Number']}")
                
                response = query_model(row['Question Text'], model)
                
                result = {
                    'Question Number': row['Question Number'],
                    'Attribute': row['Attribute'],
                    'Expertise Level': row['Expertise Level'],
                    'Question Text': row['Question Text'],
                    'LLM Name': model,
                    'Model Response': response,
                    'Score': '',  # To be filled later
                    'Explanation': ''  # To be filled later
                }
                results.append(result)
                
                # Save intermediate results with IST timestamp
                ist_time = datetime.now(ist)
                pd.DataFrame(results).to_csv(
                    f'/home/ubuntu/quantumLeap/data/eval_for_base_model_selection/temp/results_{ist_time.strftime("%Y%m%d_%H%M%S")}.csv',
                    index=False
                )
                
                time.sleep(2)  # Brief pause between questions
                
        except Exception as e:
            logging.error(f"Error during evaluation of model {model}: {e}")
        finally:
            stop_ollama_model(process)
            time.sleep(5)  # Wait before starting next model
    
    return pd.DataFrame(results)

def main():
    """Main execution function."""
    try:
        # Load questions
        questions_df = load_questions()
        
        # Run evaluations
        results_df = evaluate_models(questions_df)
        
        # Save final results with IST timestamp
        ist_time = datetime.now(ist)
        output_file = f'/home/ubuntu/quantumLeap/data/eval_for_base_model_selection/final/final_results_{ist_time.strftime("%Y%m%d_%H%M%S")}.csv'
        results_df.to_csv(output_file, index=False)
        logging.info(f"Results saved to {output_file}")
        
    except Exception as e:
        logging.error(f"Error in main execution: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd

df = pd.read_csv('/home/ubuntu/quantumLeap/data/eval_for_base_model_selection/final/final_results_20241106_104834_v2.csv')
df.head()

In [ ]:
df.shape

In [5]:
import requests
import json
from typing import Optional, Dict, Any
import logging
import sys

class OllamaAPIClient:
    """
    A client for interacting with the Ollama API with proper error handling and response parsing.
    """
    def __init__(self, base_url: str = "http://localhost:11434"):
        self.base_url = base_url.rstrip('/')
        self.session = requests.Session()
        self.session.headers.update({
            "Content-Type": "application/json",
        })
        
        # Setup logging
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO)

    def generate_text(self, 
                     model: str, 
                     prompt: str, 
                     stream: bool = True,  # Changed default to True for streaming
                     **kwargs) -> Dict[str, Any]:
        """
        Generate text using the specified model.
        
        Args:
            model: Name of the model to use
            prompt: Input prompt for generation
            stream: Whether to stream the response
            **kwargs: Additional parameters to pass to the API
            
        Returns:
            Dict containing the response from the API
        """
        endpoint = f"{self.base_url}/api/generate"
        
        payload = {
            "model": model,
            "prompt": prompt,
            "stream": stream,
            **kwargs
        }
        
        try:
            response = self.session.post(endpoint, json=payload, stream=True)
            response.raise_for_status()
            
            # Always use streaming for better token handling
            return self._handle_streaming_response(response)
            
        except requests.exceptions.RequestException as e:
            self.logger.error(f"API request failed: {e}")
            raise
            
    def _handle_streaming_response(self, response: requests.Response) -> Dict[str, Any]:
        """Handle streaming response from the API with improved token formatting."""
        combined_response = ""
        current_line = ""
        
        for line in response.iter_lines():
            if line:
                try:
                    json_response = json.loads(line)
                    if "response" in json_response:
                        token = json_response["response"]
                        current_line += token
                        # Print the token without newline and flush immediately
                        print(token, end='', flush=True)
                        combined_response += token
                        
                except json.JSONDecodeError as e:
                    self.logger.warning(f"Failed to parse streaming JSON: {e}")
                    continue
                
        # Print final newline
        print()
        return {"response": combined_response}

def main():
    client = OllamaAPIClient()
    
    prompt = """
    Envision a groundbreaking theory that unites Dr. Clotaire Rapaille's 'The Culture Code' 
    with advanced artificial intelligence and machine learning technologies to predict and 
    influence consumer behavior in the era of globalization and cultural hybridization.
    """
    
    try:
        print("\nGenerating response...\n")
        result = client.generate_text(
            # model="hf.co/RichardErkhov/Qwen_-_Qwen2.5-72B-Instruct-gguf:Q3_K",
            model="hf.co/bartowski/Llama-3.1-Nemotron-70B-Instruct-HF-GGUF:Q3_K_XL ",
            prompt=prompt
        )
        # No need to print the result again since it's already printed during streaming
        
    except Exception as e:
        print(f"Error generating text: {e}")

if __name__ == "__main__":
    main()

ERROR:__main__:API request failed: 404 Client Error: Not Found for url: http://localhost:11434/api/generate



Generating response...

Error generating text: 404 Client Error: Not Found for url: http://localhost:11434/api/generate


In [ ]:

Generating response...

### The Culture-AI Fusion Theory: Predicting and Influencing Consumer Behavior in a Globalized World

#### Introduction
In an increasingly globalized world, understanding consumer behavior is more complex than ever. Dr. Clotaire Rapaille's "The Culture Code" provides valuable insights into how cultural archetypes and collective unconscious influence consumer decisions. However, the rapid evolution of artificial intelligence (AI) and machine learning (ML) technologies offers a new dimension to these insights, enabling us to predict and influence consumer behavior with unprecedented accuracy and nuance.

#### The Core Idea
The Culture-AI Fusion Theory integrates Dr. Rapaille's cultural codes with advanced AI and ML techniques to create a comprehensive model for understanding and predicting consumer behavior. This theory leverages the following components:

1. **Cultural Archetypes**: Drawing from Dr. Rapaille's work, we identify and codify cultural archetypes that are deeply embedded in various societies. These archetypes represent fundamental beliefs, values, and behaviors that shape consumer decisions.

2. **Data Collection and Analysis**: Advanced AI and ML algorithms process vast amounts of data from diverse sources, including social media, online transactions, and cultural content (e.g., literature, films). This data is used to identify patterns and trends in consumer behavior across different cultures.

3. **Predictive Modeling**: Machine learning models are trained to predict consumer behavior based on the identified cultural archetypes and real-time data. These models can forecast how consumers from different cultural backgrounds will respond to various marketing strategies, product designs, and brand messages.

4. **Behavioral Influence**: By understanding the underlying cultural codes, AI systems can generate personalized marketing content that resonates with specific cultural values and preferences. This approach not only enhances consumer engagement but also builds trust and loyalty.

#### Key Components of the Theory

1. **Cultural Code Identification**
   - **Archetype Extraction**: Using natural language processing (NLP) and sentiment analysis, AI algorithms extract key archetypes from cultural content.
   - **Code Mapping**: Cultural codes are mapped to specific consumer behaviors and preferences using a combination of supervised and unsupervised learning techniques.

2. **Data Integration and Processing**
   - **Multisource Data Collection**: Data is collected from various sources, including social media platforms, e-commerce websites, and cultural databases.
   - **Real-time Data Analysis**: AI models process real-time data to capture the dynamic nature of consumer behavior in a globalized context.

3. **Predictive Modeling**
   - **Behavioral Prediction**: Machine learning algorithms predict how consumers from different cultural backgrounds will respond to marketing strategies and product offerings.
   - **Scenario Simulation**: Simulate various marketing scenarios to identify the most effective strategies for each cultural group.

4. **Behavioral Influence**
   - **Personalized Marketing**: AI systems generate personalized content that aligns with specific cultural codes, enhancing consumer engagement and satisfaction.
   - **Cultural Sensitivity**: Ensure that marketing messages are culturally sensitive and respectful, building trust and loyalty among diverse consumer groups.

#### Application Examples

1. **Global Branding Strategies**
   - **Cultural Code Alignment**: A global brand can use AI to align its messaging with the cultural codes of different regions, ensuring that its branding resonates with local consumers.
   - **Adaptive Marketing Campaigns**: AI-driven campaigns adapt in real-time based on consumer feedback and cultural trends.

2. **Product Design**
   - **Culturally Relevant Designs**: AI can help design products that are culturally relevant and appealing to specific markets, reducing the risk of market failure.
   - **User-Centric Innovations**: Machine learning algorithms can identify emerging cultural trends and incorporate them into product development.

3. **Customer Experience**
   - **Personalized Customer Service**: AI-powered chatbots and customer service systems can provide personalized assistance that aligns with cultural expectations.
   - **Cultural Training for Employees**: AI can help train employees on cultural sensitivity, improving the overall customer experience.

#### Conclusion
The Culture-AI Fusion Theory represents a revolutionary approach to understanding and influencing consumer behavior in a globalized world. By combining Dr. Clotaire Rapaille's insights into cultural codes with advanced AI and ML technologies, businesses can gain a deeper understanding of their consumers and create more effective marketing strategies. This theory not only enhances consumer engagement and satisfaction but also fosters cultural sensitivity and respect, building stronger relationships between brands and diverse consumer groups.
